In [ ]:
#!pip install langchain_community
#!pip install pandas
#!pip install numpy

In [ ]:
from langchain_community.llms import Ollama
import pandas as pd


In [ ]:
llm = Ollama(model = "llama2")


In [ ]:
### Read transaction data from the bank statement 
df = pd.read_csv('statement.csv')
df


In [ ]:
### Get unique transaction descriptions / name column
unique_transactions = df['Name / Description'].unique()
len(unique_transactions)

unique_transactions[1: 10]

In [ ]:
### Categorize bank transaction with llama2
def hop(start, stop, step):
    for i in range (start, stop, step):
        yield i
    yield stop
index_list = list(hop(0, len(unique_transactions), 30))
index_list



In [19]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all


In [93]:
categories_df_all.to_csv('categories.csv', index=False)

In [ ]:
### Get unique categories
unique_categories = categories_df_all['Category'].unique()
unique_categories

In [ ]:
### Drop the rows with NaN values
categories_df_all = categories_df_all.dropna()
categories_df_all   

In [96]:
# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
categories_df_all

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

In [ ]:
# Merge the categories_df_all with the statement.csv dataframe (df)
import re
categories_df_all["Transaction"] = categories_df_all["Transaction"].apply(lambda x: re.sub(r"^\d+\.\s+", "", x))
df["Name / Description"] = df["Name / Description"].str.lower().str.strip()
categories_df_all["Transaction"] = categories_df_all["Transaction"].str.lower().str.strip()
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df.to_csv("statement_categorized.csv", index=False)
